# Outlier Detection

This notebook will walk you through a workflow for outlier detection.

## TOC

 * [Detecting outlier jobs](#detect-outlier-jobs-study-1)
  * [partition jobs](#partition-jobs)
  * [create a trained model](#create-ref-model)

## <a name="detect-outlier-jobs-study-1">Case Study 1 - Detecting outlier jobs</a>

In this example we will use synthetic data. The data was generated using:
```
sample/outlier/workload.sh
```

The script basically compiles the linux kernel a few times. For certain compiles
it adds a background workload, so the compile takes longer. The slower job is marked
with the an 'outlier' suffix. However, we will pretend we don't know the outlier
and figure it out.

Along the way, we will also learn how to create a trained model, and use it 
for future outlier detection.


### Requirements

You will need to import data lying in `sample/outlier/*.tgz`:
```
$ ./epmt -v submit sample/outlier/*.tgz


In [1]:
import epmt_query as eq
import epmt_outliers as eod

INFO:epmt_job:Binding to DB: {'host': 'localhost', 'user': 'postgres', 'password': 'example', 'dbname': 'EPMT', 'provider': 'postgres'}
INFO:epmt_job:Generating mapping from schema...


{'host': 'localhost', 'user': 'postgres', 'password': 'example', 'dbname': 'EPMT', 'provider': 'postgres'}


In [2]:
jobs = eq.get_jobs(tag='exp_name:linux_kernel', fmt='terse')
jobs

['kern-6656-20190614-185359',
 'kern-6656-20190614-190245',
 'kern-6656-20190614-191138',
 'kern-6656-20190614-192044-outlier',
 'kern-6656-20190614-194024',
 'kern-6656-20190614-194953',
 'kern-6656-20190614-195909',
 'kern-6656-20190614-200819',
 'kern-6656-20190614-201744-outlier']

In [3]:
# As a first pass let's see whether the outliers can be auto-detected
(df, fdict) = eod.detect_outlier_jobs(jobs)
df

,jobid,duration,cpu_time,num_procs
0,kern-6656-20190614-185359,0,1,0
1,kern-6656-20190614-190245,0,0,0
2,kern-6656-20190614-191138,0,0,0
3,kern-6656-20190614-192044-outlier,1,1,0
4,kern-6656-20190614-194024,0,0,0
5,kern-6656-20190614-194953,0,0,0
6,kern-6656-20190614-195909,0,0,0
7,kern-6656-20190614-200819,0,0,0
8,kern-6656-20190614-201744-outlier,1,1,0


As you can see, while we did catch both the outliers, there
is also the "false positive" on one "non-outlier" process
The reason the 1 is marked for `duration` and `cpu_time` but `not num_procs`
is because the background compute process increased the job duration
but not the number of sub-processes of our workload.

In [4]:
fdict

{'cpu_time': ({'kern-6656-20190614-190245',
   'kern-6656-20190614-191138',
   'kern-6656-20190614-194024',
   'kern-6656-20190614-194953',
   'kern-6656-20190614-195909',
   'kern-6656-20190614-200819'},
  {'kern-6656-20190614-185359',
   'kern-6656-20190614-192044-outlier',
   'kern-6656-20190614-201744-outlier'}),
 'duration': ({'kern-6656-20190614-185359',
   'kern-6656-20190614-190245',
   'kern-6656-20190614-191138',
   'kern-6656-20190614-194024',
   'kern-6656-20190614-194953',
   'kern-6656-20190614-195909',
   'kern-6656-20190614-200819'},
  {'kern-6656-20190614-192044-outlier', 'kern-6656-20190614-201744-outlier'}),
 'num_procs': ({'kern-6656-20190614-185359',
   'kern-6656-20190614-190245',
   'kern-6656-20190614-191138',
   'kern-6656-20190614-192044-outlier',
   'kern-6656-20190614-194024',
   'kern-6656-20190614-194953',
   'kern-6656-20190614-195909',
   'kern-6656-20190614-200819',
   'kern-6656-20190614-201744-outlier'},
  set())}

`fdict` the other return value is a dictionary keyed by `feature`. The value is a tuple of two partitions based on the `feature`. The first partition being the reference set, and the second partition is the outlier set.
This partitioning <a name="partition-jobs"></a> can be more simply obtained as follows:

In [5]:
parts = eod.partition_jobs(jobs, features=['duration'])
parts

{'duration': ({'kern-6656-20190614-185359',
   'kern-6656-20190614-190245',
   'kern-6656-20190614-191138',
   'kern-6656-20190614-194024',
   'kern-6656-20190614-194953',
   'kern-6656-20190614-195909',
   'kern-6656-20190614-200819'},
  {'kern-6656-20190614-192044-outlier', 'kern-6656-20190614-201744-outlier'})}

Above, we just got the partitioning of the jobs on a single `feature` -- `duration`.

Now would be a good time to <a name="create-ref-model"></a>create a trained model based on the
set of jobs in the reference partition:

In [6]:
ref_jobs = parts['duration'][0]
ref_jobs

{'kern-6656-20190614-185359',
 'kern-6656-20190614-190245',
 'kern-6656-20190614-191138',
 'kern-6656-20190614-194024',
 'kern-6656-20190614-194953',
 'kern-6656-20190614-195909',
 'kern-6656-20190614-200819'}

In [7]:
r = eq.create_refmodel(ref_jobs, tag='exp_name:linux_kernel;type:ref')

In [8]:
r['id'], r['tags']

(4, {'exp_name': 'linux_kernel', 'type': 'ref'})

We added a tag to help search for this trained/ref model later.

In [9]:
# using the trained model is as simple as:
(df, _) = eod.detect_outlier_jobs(jobs, trained_model = r['id'])
df

,jobid,duration,cpu_time,num_procs
0,kern-6656-20190614-185359,0,0,0
1,kern-6656-20190614-190245,0,0,0
2,kern-6656-20190614-191138,0,0,0
3,kern-6656-20190614-192044-outlier,1,1,0
4,kern-6656-20190614-194024,0,0,0
5,kern-6656-20190614-194953,0,0,0
6,kern-6656-20190614-195909,0,0,0
7,kern-6656-20190614-200819,0,0,0
8,kern-6656-20190614-201744-outlier,1,1,0


Obviously the jobs that were used to create the reference model will not be 
classifed as outliers for any feature.

This marks the end of this case study. In a following study we will explore how
to detect outliers in individual operations and create a trained model for ops.

## <a name="rca-jobs"></a> Case Study II - Root Cause Analysis

In this study we will do an RCA with real data generated from GFDL PP runs.

```
$ ./epmt -v submit $(cat <<EOT
> sample/ppr-batch/1854/625151.tgz
> sample/ppr-batch/1859/627907.tgz
> sample/ppr-batch/1869/633114.tgz
> sample/ppr-batch/1864/629322.tgz
> sample/ppr-batch/1884/685001.tgz
> sample/ppr-batch/1874/675992.tgz
> sample/ppr-batch/1879/680163.tgz
> sample/ppr-batch/1889/691209.tgz
> sample/ppr-batch/1894/693129.tgz
> EOT
> )
```

All these jobs share the following tags: `{u'ocn_res': u'0.5l75', u'atm_res': u'c96l49', u'exp_component': u'ocean_annual_z_1x1deg', u'exp_name': u'ESM4_historical_D151'`. The difference is only that they have different values for `('exp_time', 'script_name')`.

If you are curious how we found these comparable jobs, here is the query:
```
>>> x = Job.select(lambda j: j.tags['exp_component'] == 'ocean_annual_z_1x1deg').filter(lambda j: j.tags['exp_name'] == 'ESM4_historical_D151')
>>> eq.get_jobs(x, fmt="terse")
```

In [10]:
jobs = eq.get_jobs(tag="exp_name:ESM4_historical_D151;exp_component:ocean_annual_z_1x1deg", fmt='terse')
jobs

['625151',
 '627907',
 '633114',
 '629322',
 '685001',
 '675992',
 '680163',
 '802938',
 '691209',
 '693129',
 '696110',
 '804266']

Now, let's partition the jobs by `cpu_time`: 

In [11]:
parts = eod.partition_jobs(jobs, features=['cpu_time'])
parts

{'cpu_time': ({'627907',
   '629322',
   '633114',
   '675992',
   '680163',
   '685001',
   '691209',
   '693129',
   '696110',
   '802938',
   '804266'},
  {'625151'})}

As you can see, the first partition contains 9 conformant jobs, and the outlier
partition contains a single job `625151`. Let's see how different is the `cpu_time`
of this job v. the rest:

In [12]:
jobs_df = eq.get_jobs(jobs, fmt='pandas', order='desc(j.cpu_time)')
display(jobs_df.columns.values)
jobs_df[['jobid', 'cpu_time', 'duration', 'num_procs']]

array(['PERF_COUNT_SW_CPU_CLOCK', 'account', 'all_proc_tags',
       'cancelled_write_bytes', 'cpu_time', 'created_at',
       'delayacct_blkio_time', 'duration', 'end', 'env_changes_dict',
       'env_dict', 'exitcode', 'guest_time', 'inblock', 'info_dict',
       'invol_ctxsw', 'jobid', 'jobname', 'jobscriptname', 'majflt',
       'minflt', 'num_procs', 'num_threads', 'outblock', 'processor',
       'queue', 'rchar', 'rdtsc_duration', 'read_bytes', 'rssmax',
       'sessionid', 'start', 'submit', 'syscr', 'syscw', 'systemtime',
       'tags', 'time_oncpu', 'time_waiting', 'timeslices', 'updated_at',
       'user', 'user+system', 'usertime', 'vol_ctxsw', 'wchar',
       'write_bytes'], dtype=object)

,jobid,cpu_time,duration,num_procs
0,625151,1224444629,10425623185,13530
1,627907,694594906,6589174875,4411
2,629322,622137956,7286331754,4411
3,802938,558475582,3986871458,4411
4,691209,536679993,860163243,4411
5,804266,535237219,3285839213,4411
6,693129,533686454,3619324767,4411
7,675992,520837207,9114150525,4411
8,685001,480310481,6815710476,4428
9,680163,462936089,6156192011,4411


It's really interesting that the outlier has thrice the number of processes
as the others. Let's see if we can uncover more..

In [13]:
(refs, outl) = parts['cpu_time']
(refs, outl)

({'627907',
  '629322',
  '633114',
  '675992',
  '680163',
  '685001',
  '691209',
  '693129',
  '696110',
  '802938',
  '804266'},
 {'625151'})

In [14]:
(_, df, flist) = eod.detect_rootcause(refs, '625151')
df

,num_procs,cpu_time,duration
count,1.100000e+01,1.100000e+01,1.100000e+01
mean,4.415636e+03,5.252580e+08,5.205965e+09
std,1.099339e+01,8.507230e+07,2.348081e+09
min,4.411000e+03,3.757608e+08,8.601632e+08
25%,4.411000e+03,4.716233e+08,3.567232e+09
50%,4.411000e+03,5.336865e+08,6.036720e+09
75%,4.411000e+03,5.475778e+08,6.702443e+09
max,4.445000e+03,6.945949e+08,9.114151e+09
input,1.353000e+04,1.224445e+09,1.042562e+10
ref_max_modified_z_score,inf,2.033400e+00,1.703300e+00
